## Import Libraries

In [199]:
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfTransformer,
    CountVectorizer,
)
from sklearn.preprocessing import Normalizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from collections import Counter
from bs4 import BeautifulSoup
import pandas as pd
import spacy
import re
import itertools
from langdetect import detect
import langdetect
from collections import Counter
import math
import random
import nltk
from nltk import FreqDist
from nltk.corpus import reuters
from nltk.util import bigrams
from nltk.probability import ConditionalFreqDist
from nltk import bigrams, FreqDist
from nltk.probability import LidstoneProbDist
from nltk.tokenize import word_tokenize


## Data Preprocessing

load the data

In [200]:
data = pd.read_csv("./../Q1/data.csv")

In [201]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1157 entries, 0 to 1156
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   postUrl             1157 non-null   object
 1   id                  1157 non-null   int64 
 2   text                1156 non-null   object
 3   ownerUsername       1157 non-null   object
 4   ownerProfilePicUrl  1157 non-null   object
 5   timestamp           1157 non-null   object
 6   likesCount          1157 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 63.4+ KB


In [202]:
data.head()

,postUrl,id,text,ownerUsername,ownerProfilePicUrl,timestamp,likesCount
0,https://www.instagram.com/p/Cz67N84Pezn/,17981320055390052,NaN,n4i1er,https://scontent-dub4-1.cdninstagram.com/v/t51...,2023-12-09T01:46:41.000Z,0
1,https://www.instagram.com/p/Cz67N84Pezn/,18225095332247021,😍😍🔥🔥🔥,farid.zand1997,https://scontent-dub4-1.cdninstagram.com/v/t51...,2023-12-09T09:30:36.000Z,0
2,https://www.instagram.com/p/Cz67N84Pezn/,18016249762974314,patm,andreprivet_,https://scontent-dub4-1.cdninstagram.com/v/t51...,2023-12-09T12:32:20.000Z,0
3,https://www.instagram.com/p/Cz67N84Pezn/,17980123316614267,@hoccein_hemati68 این کوصکش تو ایران بود نهایت...,kurd___boy666,https://scontent-lga3-2.cdninstagram.com/v/t51...,2023-12-09T13:14:43.000Z,0
4,https://www.instagram.com/p/Cz67N84Pezn/,18036756691566765,@amir_niarashid 💩🤣🖕,kurd___boy666,https://scontent-lga3-2.cdninstagram.com/v/t51...,2023-12-09T13:15:08.000Z,0


Remove duplicate comments

In [203]:
data = data.drop_duplicates(subset=["text"])

Remove comments with empty text

In [204]:
data = data[data['text'].notna()]

Remove text that is not related to English

In [205]:
nlp = spacy.load("en_core_web_sm")

In [206]:
data = data[
    data["text"].apply(lambda x: all([token.pos_ != "FOREIGN" for token in nlp(x)]))
    == True
]

Convert timestamp to a standard format

In [207]:
data["timestamp"] = pd.to_datetime(data["timestamp"])

Remove non-ASCII characters

In [208]:
data["text"] = data["text"].apply(lambda x: "".join([ch for ch in x if ord(ch) < 128]))

In [209]:
data.head()

,postUrl,id,text,ownerUsername,ownerProfilePicUrl,timestamp,likesCount
1,https://www.instagram.com/p/Cz67N84Pezn/,18225095332247021,,farid.zand1997,https://scontent-dub4-1.cdninstagram.com/v/t51...,2023-12-09 09:30:36+00:00,0
2,https://www.instagram.com/p/Cz67N84Pezn/,18016249762974314,patm,andreprivet_,https://scontent-dub4-1.cdninstagram.com/v/t51...,2023-12-09 12:32:20+00:00,0
3,https://www.instagram.com/p/Cz67N84Pezn/,17980123316614267,@hoccein_hemati68 !,kurd___boy666,https://scontent-lga3-2.cdninstagram.com/v/t51...,2023-12-09 13:14:43+00:00,0
4,https://www.instagram.com/p/Cz67N84Pezn/,18036756691566765,@amir_niarashid,kurd___boy666,https://scontent-lga3-2.cdninstagram.com/v/t51...,2023-12-09 13:15:08+00:00,0
5,https://www.instagram.com/p/Cz67N84Pezn/,18118091365334462,Nice arms,st3ph_3s0n,https://scontent-dub4-1.cdninstagram.com/v/t51...,2023-12-09 20:37:29+00:00,0


Convert text to lowercase

In [210]:
data["text"] = data["text"].str.lower()

Remove punctuation

In [211]:
data["text"] = data["text"].str.replace("[^\w\s]", "")

Remove stop words

In [212]:
stop_words = set(stopwords.words("english"))
data["text"] = data["text"].apply(
    lambda x: " ".join([word for word in x.split() if word not in stop_words])
)

In [213]:
data = data[data["text"].str.strip() != ""]

In [214]:
data.head()

,postUrl,id,text,ownerUsername,ownerProfilePicUrl,timestamp,likesCount
2,https://www.instagram.com/p/Cz67N84Pezn/,18016249762974314,patm,andreprivet_,https://scontent-dub4-1.cdninstagram.com/v/t51...,2023-12-09 12:32:20+00:00,0
3,https://www.instagram.com/p/Cz67N84Pezn/,17980123316614267,@hoccein_hemati68 !,kurd___boy666,https://scontent-lga3-2.cdninstagram.com/v/t51...,2023-12-09 13:14:43+00:00,0
4,https://www.instagram.com/p/Cz67N84Pezn/,18036756691566765,@amir_niarashid,kurd___boy666,https://scontent-lga3-2.cdninstagram.com/v/t51...,2023-12-09 13:15:08+00:00,0
5,https://www.instagram.com/p/Cz67N84Pezn/,18118091365334462,nice arms,st3ph_3s0n,https://scontent-dub4-1.cdninstagram.com/v/t51...,2023-12-09 20:37:29+00:00,0
7,https://www.instagram.com/p/Cz67N84Pezn/,18247697080171707,gostoso,pedr0_hgc,https://scontent-dub4-1.cdninstagram.com/v/t51...,2023-12-10 01:28:48+00:00,0


## perplexity

In [215]:
def calculate_perplexity(probabilities):
    if not isinstance(probabilities, list):
        raise TypeError("Input must be a list of probabilities.")

    if not probabilities:
        raise ValueError("The list of probabilities is empty.")

    if not all(0 <= prob <= 1 for prob in probabilities):
        raise ValueError("All probabilities must be in the range [0, 1].")

    N = len(probabilities)
    log_prob_sum = sum(math.log2(prob) for prob in probabilities if prob > 0)

    if log_prob_sum == 0:
        raise ValueError(
            "All probabilities are zero, resulting in infinite perplexity."
        )

    perplexity = 2 ** (-1 / N * log_prob_sum)

    return perplexity

## Unigram Language Model

### Introduction 

A unigram language model is a simple yet powerful statistical model used in natural language processing (NLP) to predict the probability of occurrence of a word in a sequence of words. It assumes that the probability of a word appearing in a sentence is independent of the context in which it appears. This assumption, while not entirely accurate, often provides a reasonable approximation of the true word distribution in a language.

### Implementation

In [216]:
class UnigramLanguageModel:
    def __init__(self, training_corpus, smoothing_factor=0.1):
        self.smoothing_factor = smoothing_factor
        self.word_counts = Counter(training_corpus)
        self.total_words = len(training_corpus)
        self.vocabulary_size = len(set(training_corpus))

    def calculate_probability(self, word):
        # Add smoothing to handle unseen words
        smoothed_probability = (self.word_counts[word] + self.smoothing_factor) / (self.total_words + self.smoothing_factor * self.vocabulary_size)
        return smoothed_probability

    def generate_sentence_with_probability(self, length=5):
        sentence = [random.choices(list(self.word_counts.keys()), weights=[self.calculate_probability(word) for word in self.word_counts.keys()])[0] for _ in range(length)]
        return sentence


In [217]:
training_data = ["the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"]

In [218]:
unigram_model = UnigramLanguageModel(training_data)

In [219]:
def generate_sentence(model, length=5):
    sentence = []
    for _ in range(length):
        # Sample a word based on its probability
        sampled_word = random.choices(list(model.word_counts.keys()), weights=[model.calculate_probability(word) for word in model.word_counts.keys()])[0]
        sentence.append(sampled_word)
    return sentence

# Generate 5 sentences
for i in range(5):
    generated_sentence = unigram_model.generate_sentence_with_probability(length=8)
    print(f"Generated Sentence: {' '.join(generated_sentence)}")


Generated Sentence: lazy fox fox brown dog dog the lazy
Generated Sentence: quick brown jumps fox quick fox dog fox
Generated Sentence: over brown lazy brown fox jumps fox brown
Generated Sentence: quick the fox fox brown jumps quick over
Generated Sentence: lazy jumps the jumps over jumps brown quick


## bigram Language Model

### Introduction

A bigram language model is a statistical language model that predicts the probability of a word appearing in a sequence of words based on the word that precedes it. Unlike unigram models, which assume that words occur independently of each other, bigram models take into account the sequential nature of language.

### Implementation

Tokenize the dataset

In [220]:
data = "".join(training_data)
tokens = word_tokenize(data)

Create bigrams

In [221]:
bi_grams = list(bigrams(tokens))

Calculate bigram frequencies

In [222]:
bigram_freq = FreqDist(bi_grams)

Add smoothing (Laplace smoothing)

In [223]:
alpha = 0.1
vocab_size = len(set(tokens))
prob_dist = LidstoneProbDist(bigram_freq, alpha, bins=vocab_size)

Generate sentences

In [250]:
import random
from collections import defaultdict

def train_bigram_model(corpus):
    bigram_counts = defaultdict(lambda: defaultdict(int))
    unigram_counts = defaultdict(int)

    # Count occurrences of unigrams and bigrams
    for sentence in corpus:
        tokens = sentence.split()
        for i in range(len(tokens) - 1):
            bigram_counts[tokens[i]][tokens[i+1]] += 1
            unigram_counts[tokens[i]] += 1

    return bigram_counts, unigram_counts

def generate_sentence(bigram_model, start_token, max_length=10):
    sentence = [start_token]
    current_token = start_token

    for _ in range(max_length):
        next_tokens = list(bigram_model[current_token].keys())
        if not next_tokens:
            break

        probabilities = [bigram_model[current_token][token] / (unigram_counts[current_token] + len(bigram_model[current_token]))
                         for token in next_tokens]
        next_token = random.choices(next_tokens, weights=probabilities, k=1)[0]

        sentence.append(next_token)
        current_token = next_token

    return ' '.join(sentence)

# Example usage
corpus = [
    "This is a sample sentence.",
    "Sample sentence for language model.",
    "Language model with bigram smoothing.",
    "Smoothing helps handle unseen bigrams.",
    "Bigrams and smoothing in natural language processing."
]

bigram_model, unigram_counts = train_bigram_model(corpus)

# Generate 5 sentences
for _ in range(5):
    generated_sentence = generate_sentence(bigram_model, "Language")
    print(generated_sentence)

Language model with bigram smoothing.
Language model with bigram smoothing.
Language model with bigram smoothing.
Language model with bigram smoothing.
Language model with bigram smoothing.


In [232]:
seed_word = random.choice(tokens)

Generate 5 sentences

In [237]:
generated_sentences = generate_sentence(seed_word)

TypeError: ProbDistI.generate() takes 1 positional argument but 2 were given

Print the generated sentences

In [191]:
for i, sentence in enumerate(generated_sentences, start=1):
    print(f"Sentence {i}: {sentence}")

Sentence 1: thequickbrownfoxjumpsoverthelazydog
Sentence 2: thequickbrownfoxjumpsoverthelazydog
Sentence 3: thequickbrownfoxjumpsoverthelazydog
Sentence 4: thequickbrownfoxjumpsoverthelazydog
Sentence 5: thequickbrownfoxjumpsoverthelazydog


## trigram Language Model

### Introduction

A trigram language model is a statistical language model that predicts the probability of a word appearing in a sequence of words based on the two preceding words. It takes into account the sequential nature of language by considering the dependencies between three consecutive words.

### Implementation

Create a CountVectorizer for trigrams

In [44]:
vectorizer = CountVectorizer(ngram_range=(3, 3))

Fit and transform the text data

In [45]:
X = vectorizer.fit_transform(data["text"])

Get the feature names (trigrams)

In [46]:
feature_names = vectorizer.get_feature_names_out()

Get the counts for each trigram

In [47]:
trigram_counts = X.sum(axis=0)

Create a DataFrame to display the trigram counts

In [48]:
trigram_df = pd.DataFrame({"Trigram": feature_names, "Count": trigram_counts.A1})

Sort the DataFrame by trigram counts

In [49]:
trigram_df = trigram_df.sort_values(by="Count", ascending=False)

Display the top trigrams

In [50]:
trigram_df.head(10)

,Trigram,Count
2092,se quiser sim,13
1988,quiser sim mano,11
1321,lembra muito meu,3
1769,pensou em competir,3
1320,lembra meu comeo,3
1945,que los anabolicos,2
307,cabeza de bolo,2
2611,work people use,2
62,activates hormone destroys,2
2367,told would work,2


### trigram smoothing

Create a CountVectorizer for trigrams

In [51]:
vectorizer = CountVectorizer(ngram_range=(3, 3))

Fit and transform the text data

In [52]:
X = vectorizer.fit_transform(data["text"])

Get the feature names (trigrams)

In [53]:
feature_names = vectorizer.get_feature_names_out()

Get the counts for each trigram

In [54]:
trigram_counts = X.sum(axis=0)

Add-one (Laplace) smoothing

In [55]:
total_trigrams = len(feature_names)
smoothed_counts = trigram_counts + 1
total_tokens = smoothed_counts.sum()

Calculate smoothed probabilities

In [56]:
smoothed_probabilities = smoothed_counts / total_tokens

Create a DataFrame to display the smoothed trigram probabilities

In [57]:
smoothed_trigram_model = pd.DataFrame(
    {
        "Trigram": feature_names,
        "Count": smoothed_counts.A1,
        "Probability": smoothed_probabilities.A1,
    }
)

Sort the DataFrame by trigram counts

In [58]:
smoothed_trigram_model = smoothed_trigram_model.sort_values(by="Count", ascending=False)

Display the top trigrams with smoothed probabilities

In [59]:
smoothed_trigram_model.head(10)

,Trigram,Count,Probability
2092,se quiser sim,14,0.002581
1988,quiser sim mano,12,0.002212
1321,lembra muito meu,4,0.000737
1769,pensou em competir,4,0.000737
1320,lembra meu comeo,4,0.000737
1945,que los anabolicos,3,0.000553
307,cabeza de bolo,3,0.000553
2611,work people use,3,0.000553
62,activates hormone destroys,3,0.000553
2367,told would work,3,0.000553
